In [144]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [145]:
#laptop
#MCF_DS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
MCF_DS = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt",delimiter="\ ", engine='python', index_col=0)

In [146]:
#laptop
#HCC_DS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
HCC_DS = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)

In [147]:
MCF_SS = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\SmartSeq\\MCF7_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)
HCC_SS = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\SmartSeq\\HCC1806_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)

In [148]:
#function to create a y column as a separate dataframe
#1 is Normoxia, 0 is Hypoxia
def oxia(df):
    new_row = {col: None for col in df.columns}
    for col in df.columns:
        if '_Nor' in col:
            new_row[col] = 1
        elif '_Hyp' in col:
            new_row[col] = 0
    return pd.DataFrame(new_row, index=['oxia'])

In [149]:
#Oxia them and split them into X and y
y_MCF_DS = oxia(MCF_DS).T
y_HCC_DS = oxia(HCC_DS).T
y_MCF_SS = oxia(MCF_SS)
y_HCC_SS = oxia(HCC_SS)

In [150]:
MCF_DS = MCF_DS.T
HCC_DS = HCC_DS.T
HCC_SS = HCC_SS.T
MCF_SS = MCF_SS.T
y_MCF_SS = y_MCF_SS.T
y_HCC_SS = y_HCC_SS.T

In [151]:
#split X and y into training and testing
X_train_MCF_DS, X_test_MCF_DS, y_train_MCF_DS, y_test_MCF_DS = train_test_split(MCF_DS, y_MCF_DS, test_size=0.3)
X_train_HCC_DS, X_test_HCC_DS, y_train_HCC_DS, y_test_HCC_DS = train_test_split(HCC_DS, y_HCC_DS, test_size=0.3)
X_train_MCF_SS, X_test_MCF_SS, y_train_MCF_SS, y_test_MCF_SS = train_test_split(MCF_SS, y_MCF_SS, test_size=0.3)
X_train_HCC_SS, X_test_HCC_SS, y_train_HCC_SS, y_test_HCC_SS = train_test_split(HCC_SS, y_HCC_SS, test_size=0.3)

In [152]:
#split testing into testing and validation
X_test_MCF_DS, X_val_MCF_DS, y_test_MCF_DS, y_val_MCF_DS = train_test_split(X_test_MCF_DS, y_test_MCF_DS, test_size=0.5)
X_test_HCC_DS, X_val_HCC_DS, y_test_HCC_DS, y_val_HCC_DS = train_test_split(X_test_HCC_DS, y_test_HCC_DS, test_size=0.5)
X_test_MCF_SS, X_val_MCF_SS, y_test_MCF_SS, y_val_MCF_SS = train_test_split(X_test_MCF_SS, y_test_MCF_SS, test_size=0.5)
X_test_HCC_SS, X_val_HCC_SS, y_test_HCC_SS, y_val_HCC_SS = train_test_split(X_test_HCC_SS, y_test_HCC_SS, test_size=0.5)

In [153]:
#put all the data into a dictionary
data = {'MCF_DS': [X_train_MCF_DS, X_test_MCF_DS, y_train_MCF_DS, y_test_MCF_DS], 'HCC_DS': [X_train_HCC_DS, X_test_HCC_DS, y_train_HCC_DS, y_test_HCC_DS], 'MCF_SS': [X_train_MCF_SS, X_test_MCF_SS, y_train_MCF_SS, y_test_MCF_SS], 'HCC_SS': [X_train_HCC_SS, X_test_HCC_SS, y_train_HCC_SS, y_test_HCC_SS]}

In [156]:
#dictionary of models:
models = {'Logistic Regression': LogisticRegression(), 'KNN': KNeighborsClassifier(), 'Random Forest': RandomForestClassifier(), 'SVM': SVC(), 'Neural Network': MLPClassifier(), 'Linear Regression': LinearRegression()}

In [157]:
#now do this with a grid search to find optimal parameteters
#create a dictionary of parameters for each model
params = {'Logistic Regression': {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
          'KNN': {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']},
          'Random Forest': {'n_estimators': [10, 50, 100, 200], 'max_depth': [10, 50, 100, 200]},
          'SVM': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
          'Neural Network': {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)], 'learning_rate_init': [0.1, 0.01, 0.001]},
          'Linear Regression': {'fit_intercept': [True, False], 'normalize': [True, False]}
          }

In [159]:
#print best parameters
#store the best parameters for each dataset and model in a dictionary
best_params = {}
for key in data:
    for model in models:
        grid = GridSearchCV(models[model], params[model], cv=5)
        grid.fit(data[key][0], np.ravel(data[key][2]))
        y_pred = grid.predict(data[key][1])
        best_params[key + model] = grid.best_params_
        print(model, key, accuracy_score(data[key][3], y_pred), classification_report(data[key][3], y_pred))
        print(grid.best_params_)

C:\Users\Cameron\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Cameron\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

KeyboardInterrupt: 

In [ ]:
print(best_params)